In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Breast_Cancer/GSE249377'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Exploring the Effects of Experimental Parameters and Data Modeling Approaches on In Vitro Transcriptomic Point-of-Departure Estimates"
!Series_summary	"Multiple new approach methods (NAMs) are being developed to rapidly screen large numbers of chemicals to aid in hazard evaluation and risk assessments. High-throughput transcriptomics (HTTr) in human cell lines has been proposed as a first-tier screening approach for determining the types of bioactivity a chemical can cause (activation of specific targets vs. generalized cell stress) and for calculating transcriptional points of departure (tPODs) based on changes in gene expression. In the present study, we examine a range of computational methods to calculate tPODs from HTTr data, using six data sets in which MCF7 cells cultured in two different media formulations were treated with a panel of 44 chemicals for 3 different exposure durations (6, 12, 24 hr)."
!Series_overall_design	"Multiple computat

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

def convert_trait(value):
    try:
        return value.split(":")[1].strip() == "MCF7"  # Returns True if MCF7, False otherwise
    except:
        return None

def convert_age(value):
    try:
        return float(value.split(":")[1].strip())
    except:
        return None

def convert_gender(value):
    try:
        gender_str = value.split(":")[1].strip().lower()
        if gender_str == 'female':
            return 0
        elif gender_str == 'male':
            return 1
    except:
        return None

save_cohort_info('GSE249377', './preprocessed/Breast_Cancer/cohort_info.json', is_gene_available, trait_row is not None)
